In [ ]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 150)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_x = pd.read_csv("train_x.csv")
test_x = pd.read_csv("test_x.csv")
train_x_log = pd.read_csv("train_x_log.csv")
test_x_log = pd.read_csv("test_x_log.csv")
train_x_eng_with_KNN_TSNE_KMean = pd.read_csv("train_x_eng_with_KNN_TSNE_KMean.csv")
test_x_eng_with_KNN_TSNE_Kmean = pd.read_csv("test_x_eng_with_KNN_TSNE_Kmean.csv")
train_x_log_eng_with_KNN_TSNE_KMean = pd.read_csv("train_x_log_eng_with_KNN_TSNE_KMean.csv")
test_x_log_eng_with_KNN_TSNE_Kmean = pd.read_csv("test_x_log_eng_with_KNN_TSNE_Kmean.csv")
train_y_class = pd.read_csv("train_y_class.csv")
train_y_label = pd.read_csv("train_y_label.csv")

Submission = pd.read_csv("sampleSubmission.csv")

In [ ]:
# Total dataset features to consider
print(train_x.shape)
print(test_x.shape)
print(train_x_log.shape)
print(test_x_log.shape)
print(train_x_eng_with_KNN_TSNE_KMean.shape)
print(test_x_eng_with_KNN_TSNE_Kmean.shape)
print(train_x_log_eng_with_KNN_TSNE_KMean.shape)
print(test_x_log_eng_with_KNN_TSNE_Kmean.shape)

# Label dataset
print(train_y_class.shape)
print(train_y_label.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
train_score_scaler = StandardScaler()
test_score_scaler = StandardScaler()
train_x_normalized = pd.DataFrame(train_score_scaler.fit_transform(train_x),columns = train_x.columns)
test_x_normalized = pd.DataFrame(test_score_scaler.fit_transform(test_x),columns = test_x.columns)
train_x_log_normalized = pd.DataFrame(train_score_scaler.fit_transform(train_x_log),columns = train_x_log.columns)
test_x_log_normalized = pd.DataFrame(test_score_scaler.fit_transform(test_x_log),columns = test_x_log.columns)
train_x_eng_with_KNN_TSNE_KMean_normalized = pd.DataFrame(train_score_scaler.fit_transform(train_x_eng_with_KNN_TSNE_KMean),columns = train_x_eng_with_KNN_TSNE_KMean.columns)
test_x_eng_with_KNN_TSNE_Kmean_normalized = pd.DataFrame(test_score_scaler.fit_transform(test_x_eng_with_KNN_TSNE_Kmean),columns = test_x_eng_with_KNN_TSNE_Kmean.columns)
train_x_log_eng_with_KNN_TSNE_KMean_normalized = pd.DataFrame(train_score_scaler.fit_transform(train_x_log_eng_with_KNN_TSNE_KMean),columns = train_x_log_eng_with_KNN_TSNE_KMean.columns)
test_x_log_eng_with_KNN_TSNE_Kmean_normalized = pd.DataFrame(test_score_scaler.fit_transform(test_x_log_eng_with_KNN_TSNE_Kmean),columns = test_x_log_eng_with_KNN_TSNE_Kmean.columns)

In [ ]:
# Shuffle training data
train_y_class_columns = train_y_class.columns
training_data_x = pd.concat([train_x_normalized,train_y_class],axis=1)
training_data_x = training_data_x.sample(frac=1)
training_data_x_log = pd.concat([train_x_log_normalized,train_y_class],axis=1)
training_data_x_log = training_data_x_log.sample(frac=1)
training_data_x_eng = pd.concat([train_x_eng_with_KNN_TSNE_KMean_normalized,train_y_class],axis=1)
training_data_x_eng = training_data_x_eng.sample(frac=1)
training_data_x_log_eng = pd.concat([train_x_log_eng_with_KNN_TSNE_KMean_normalized,train_y_class],axis=1)
training_data_x_log_eng = training_data_x_log_eng.sample(frac=1)

In [ ]:
train_x_normalized = training_data_x.drop(train_y_class_columns,axis=1)
train_y_class_x = training_data_x[train_y_class_columns]
train_x_log_normalized = training_data_x_log.drop(train_y_class_columns,axis=1)
train_y_class_x_log = training_data_x_log[train_y_class_columns]
train_x_eng_with_KNN_TSNE_KMean_normalized = training_data_x_eng.drop(train_y_class_columns,axis=1)
train_y_class_x_eng = training_data_x_eng[train_y_class_columns]
train_x_log_eng_with_KNN_TSNE_KMean_normalized = training_data_x_log_eng.drop(train_y_class_columns,axis=1)
train_y_class_x_log_eng = training_data_x_log_eng[train_y_class_columns]


In [ ]:
# Total dataset features to consider
print(train_x_normalized.shape)
print(test_x_normalized.shape)
print(train_x_log_normalized.shape)
print(test_x_log_normalized.shape)
print(train_x_eng_with_KNN_TSNE_KMean_normalized.shape)
print(test_x_eng_with_KNN_TSNE_Kmean_normalized.shape)
print(train_x_log_eng_with_KNN_TSNE_KMean_normalized.shape)
print(test_x_log_eng_with_KNN_TSNE_Kmean_normalized.shape)

# Label dataset
print(train_y_class_x.shape)
print(train_y_class_x_log.shape)
print(train_y_class_x_eng.shape)
print(train_y_class_x_log_eng.shape)


# Neural Network

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

In [ ]:
# Another method to reset the model weights
# model.save_weights('model_reset_weights.h5')
# model.load_weights('model_reset_weights.h5')

# Regularization layer
# model.add(Dense(int(number_of_neurons), activation = 'relu', kernel_regularizer=regularizers.l2(0.001)))
# model.add(Dropout(0.5))

## To have reproducible result with keras

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
backend.set_session(sess)

# Reset_Model trained Weights

In [ ]:
def reset_weights(model):
    session = backend.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

In [ ]:
reset_weights(model)

In [ ]:
tf.keras.backend.clear_session()

# 1. Neural Network with train_x, train_y_class

In [ ]:
# define model
model = Sequential()
model.add(Dense(units=256,activation = "relu", input_dim=93))
model.add(BatchNormalization())
model.add(Dense(units=128,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=64,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=32,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(9, activation = "softmax"))

In [ ]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False, decay = 0.001)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=100,baseline=None)
model.compile(optimizer=adam, loss='categorical_crossentropy') #loss = loss function
print(backend.eval(model.optimizer.lr)) # print model learning rate

In [ ]:
model.fit(train_x_normalized,train_y_class_x,
          batch_size=(int(train_x.shape[0]*0.2)),
          epochs=5000,verbose=1, callbacks=[early_stop])

# Visualization

In [ ]:
loss_per_epoch = pd.DataFrame(model.history.history)
loss_per_epoch.iplot(y=["loss"], title = "loss_per_epoch")

# Pre-check testing data score with this Neural Network models if ok then proceed to Ensemble model prediction

In [ ]:
Neural_Network_train_x_precheck_score = pd.DataFrame(model.predict(test_x_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_precheck_score

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_precheck_score.csv",index=False)

# train_x Prediction for Ensemble Model

In [ ]:
Neural_Network_train_x_pred = pd.DataFrame(model.predict(train_x_normalized),columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'])

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Neural_Network_train_x_pred.to_csv("Neural_Network_train_x_pred.csv",index=False)

# test_x Prediction for Ensemble Model

In [ ]:
Neural_Network_test_x_pred = pd.DataFrame(model.predict(test_x_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_test_x_pred

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_test_x_pred.csv",index=False)

# Saving and Loading Models

In [ ]:
model.save('Neural_Network_train_x.h5')

## Load a model

In [ ]:
from keras.models import load_model
Load_model = load_model('Neural_Network_train_x.h5')

In [ ]:
Load_model.summary()

In [ ]:
Neural_Network_train_x_full_evaluate = pd.DataFrame(Load_model.predict(test_x))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_full_evaluate

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_full_evaluate.csv",index=False)

# 2. Neural Network with train_x_log, train_y_class

In [ ]:
# define model
model = Sequential()
model.add(Dense(units=256,activation = "relu", input_dim=93))
model.add(BatchNormalization())
model.add(Dense(units=128,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=64,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=32,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(9, activation = "softmax"))

In [ ]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False, decay = 0.001)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=100,baseline=None)
model.compile(optimizer=adam, loss='categorical_crossentropy') #loss = loss function
print(backend.eval(model.optimizer.lr)) # print model learning rate

In [ ]:
model.fit(train_x_log_normalized,train_y_class_x_log,
          batch_size=(int(train_x.shape[0]*0.2)),
          epochs=5000,verbose=1, callbacks=[early_stop])

# Visualization

In [ ]:
loss_per_epoch = pd.DataFrame(model.history.history)
loss_per_epoch.iplot(y=["loss","val_loss"], title = "loss_per_epoch")

# Pre-check testing data score with this Neural Network models if ok then proceed to Ensemble model prediction

In [ ]:
Neural_Network_train_x_log_precheck_score = pd.DataFrame(model.predict(test_x_log_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_log_precheck_score

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_log_precheck_score.csv",index=False)

# train_x Prediction for Ensemble Model

In [ ]:
Neural_Network_train_x_log_pred = pd.DataFrame(model.predict(train_x_log_normalized),columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'])

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Neural_Network_train_x_log_pred.to_csv("Neural_Network_train_x_log_pred.csv",index=False)

# test_x Prediction for Ensemble Model

In [ ]:
Neural_Network_test_x_log_pred = pd.DataFrame(model.predict(test_x_log_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_test_x_log_pred

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_test_x_log_pred.csv",index=False)

# Saving and Loading Models

In [ ]:
model.save('Neural_Network_train_x_log.h5')

## Load a model

In [ ]:
from keras.models import load_model
Load_model = load_model('Neural_Network_train_x_log.h5')

In [ ]:
Load_model.summary()

In [ ]:
Neural_Network_train_x_log_full_evaluate = pd.DataFrame(Load_model.predict(test_x_log))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_log_full_evaluate

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_log_full_evaluate.csv",index=False)

# 3. Neural Network with train_x_eng_with_KNN_TSNE_KMean, train_y_class

In [ ]:
# define model
model = Sequential()
model.add(Dense(units=256,activation = "relu", input_dim=227))
model.add(BatchNormalization())
model.add(Dense(units=128,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=64,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=32,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(9, activation = "softmax"))

In [ ]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False, decay = 0.001)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=100,baseline=None)
model.compile(optimizer=adam, loss='categorical_crossentropy') #loss = loss function
print(backend.eval(model.optimizer.lr)) # print model learning rate

In [ ]:
model.fit(train_x_eng_with_KNN_TSNE_KMean_normalized,train_y_class_x_eng,
          batch_size=(int(train_x.shape[0]*0.2)),
          epochs=5000,verbose=1, callbacks=[early_stop])

# Visualization

In [ ]:
loss_per_epoch = pd.DataFrame(model.history.history)
loss_per_epoch.iplot(y=["loss","val_loss"], yrange=(0,0.5), title = "loss_per_epoch")

# Pre-check testing data score with this Neural Network models if ok then proceed to Ensemble model prediction

In [ ]:
Neural_Network_train_x_eng_precheck_score = pd.DataFrame(model.predict(test_x_eng_with_KNN_TSNE_Kmean_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_eng_precheck_score

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_eng_precheck_score.csv",index=False)

# train_x Prediction for Ensemble Model

In [ ]:
Neural_Network_train_x_eng_pred = pd.DataFrame(model.predict(Neural_Network_train_x_eng_pred),columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'])

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Neural_Network_train_x_eng_pred.to_csv("Neural_Network_train_x_eng_pred.csv",index=False)

# test_x Prediction for Ensemble Model

In [ ]:
Neural_Network_test_x_eng_pred = pd.DataFrame(model.predict(test_x_eng_with_KNN_TSNE_Kmean_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_test_x_eng_pred

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_test_x_eng_pred.csv",index=False)

# Saving and Loading Models

In [ ]:
model.save('Neural_Network_train_x_eng.h5')

## Load a model

In [ ]:
from keras.models import load_model
Load_model = load_model('Neural_Network_train_x_eng.h5')

In [ ]:
Load_model.summary()

In [ ]:
Neural_Network_train_x_eng_full_evaluate = pd.DataFrame(Load_model.predict(test_x_eng_with_KNN_TSNE_Kmean))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_eng_full_evaluate

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_eng_full_evaluate.csv",index=False)

# 4. Neural Network with train_x_log_eng_with_KNN_TSNE_KMean, train_y_class

In [ ]:
# define model
model = Sequential()
model.add(Dense(units=256,activation = "relu", input_dim=227))
model.add(BatchNormalization())
model.add(Dense(units=128,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=64,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(units=32,activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(9, activation = "softmax"))

In [ ]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False, decay = 0.001)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=100,baseline=None)
model.compile(optimizer=adam, loss='categorical_crossentropy') #loss = loss function
print(backend.eval(model.optimizer.lr)) # print model learning rate

In [ ]:
model.fit(train_x_log_eng_with_KNN_TSNE_KMean_normalized,train_y_class_x_log_eng,
          batch_size=(int(train_x.shape[0]*0.2)),
          epochs=5000,verbose=1, callbacks=[early_stop])

# Visualization

In [ ]:
loss_per_epoch = pd.DataFrame(model.history.history)
loss_per_epoch.iplot(y=["loss","val_loss"], title = "loss_per_epoch")

# Pre-check testing data score with this Neural Network models if ok then proceed to Ensemble model prediction

In [ ]:
Neural_Network_train_x_log_eng_precheck_score = pd.DataFrame(model.predict(test_x_log_eng_with_KNN_TSNE_Kmean_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_log_eng_precheck_score

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_log_eng_precheck_score.csv",index=False)

# train_x Prediction for Ensemble Model

In [ ]:
Neural_Network_train_x_log_eng_pred = pd.DataFrame(model.predict(Neural_Network_train_x_log_eng_pred),columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'])

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Neural_Network_train_x_log_eng_pred.to_csv("Neural_Network_train_x_log_eng_pred.csv",index=False)

# test_x Prediction for Ensemble Model

In [ ]:
Neural_Network_test_x_log_eng_pred = pd.DataFrame(model.predict(test_x_log_eng_with_KNN_TSNE_Kmean_normalized))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_test_x_log_eng_pred

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_test_x_log_eng_pred.csv",index=False)

# Saving and Loading Models

In [ ]:
model.save('Neural_Network_train_x_log_eng.h5')

## Load a model

In [ ]:
from keras.models import load_model
Load_model = load_model('Neural_Network_train_x_log_eng.h5')

In [ ]:
Load_model.summary()

In [ ]:
Neural_Network_train_x_log_eng_full_evaluate = pd.DataFrame(Load_model.predict(test_x_log_eng_with_KNN_TSNE_Kmean))

In [ ]:
Submission[['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9']] = Neural_Network_train_x_log_eng_full_evaluate

In [ ]:
## Creating a Submission File to submit to Kaggle competition ##
Submission.to_csv("Neural_Network_train_x_log_eng_full_evaluate.csv",index=False)